In [24]:
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import math

In [25]:
def getQuarters(curDate, prevDate):
    
    prevMon = prevDate.strftime("%b")
    curMon = curDate.strftime("%b")
    prevYear = prevDate.strftime("%Y")
    curYear = curDate.strftime("%Y")
    curQuarter = f"{curMon} {curYear}"
    prevQuarter = f"{prevMon} {prevYear}"
    return curQuarter, prevQuarter

def quarterList():
    lst = []
    date = datetime.datetime(2022,12,1)
    minDate = datetime.datetime(2017,1,1)
    while(date > minDate):
        prevDate = date-relativedelta(months=3)
        curQuarter, prevQuarter = getQuarters(date, prevDate)
        lst.append([curQuarter, prevQuarter])
        date = prevDate
    return lst



In [26]:
quarterPairs = quarterList()
quarter_result_dates = pd.read_csv("C:\\Users\\garya\\ML projects\\Stock analysis\\datasets\\quarterlyResultDates.csv", index_col= False)

In [27]:
quarterly_df = pd.read_csv("C:\\Users\\garya\\ML projects\\Stock analysis\\datasets\\quarterlyResultsNSE.csv").set_index('symbol')

In [28]:
quarter_result_dates = quarter_result_dates.fillna("")
quarter_result_dates = quarter_result_dates[quarter_result_dates['nsesymbol'] != '']
quarter_result_dates = quarter_result_dates.reset_index()

In [29]:
print(quarter_result_dates.loc[0])

index                    1
companyname    amjumbobags
companycode           JB03
nsesymbol          AMJUMBO
bsesymbol                 
                  ...     
Mar 1999                  
Sep 1997                  
Mar 1998                  
Mar 1997                  
Dec 1996                  
Name: 0, Length: 110, dtype: object


In [30]:
temp_data = quarter_result_dates.loc[0]

In [31]:
temp_data = temp_data.dropna()
temp_data

index                    1
companyname    amjumbobags
companycode           JB03
nsesymbol          AMJUMBO
bsesymbol                 
                  ...     
Mar 1999                  
Sep 1997                  
Mar 1998                  
Mar 1997                  
Dec 1996                  
Name: 0, Length: 110, dtype: object

In [56]:
#   0         0          0         0
#   8         4          2         1
#(profit) (revenue)  (Ebitda) `(margin)
dict1 = {}
def getBit(str1, str2):
    if(math.isnan(str1)  or math.isnan(str2)):
        return "hello"
    a = float(str1)
    b = float(str2)
    change = a - b
    return round(change,0)

def generateCategory(prevQuarter, curQuarter, fundamental_row, symbol):
    try:
        ans = 0
        
        marginBit = getBit(fundamental_row[f"Margin {curQuarter}"], fundamental_row[f"Margin {prevQuarter}"])
        ebitdaBit = getBit(fundamental_row[f"EBITDA {curQuarter}"], fundamental_row[f"EBITDA {prevQuarter}"])
        revenueBit = getBit(fundamental_row[f"Revenue {curQuarter}"], fundamental_row[f"Revenue {prevQuarter}"])
        profitBit = getBit(fundamental_row[f"Profit {curQuarter}"], fundamental_row[f"Profit {prevQuarter}"])

        if(dict1.get(str([marginBit, ebitdaBit,revenueBit, profitBit]), "default") == "default"):
            dict1[str([marginBit, ebitdaBit,revenueBit, profitBit])] = set()
        dict1[str([marginBit, ebitdaBit,revenueBit, profitBit])].add(str([symbol, curQuarter]))
        if(ans > 15):
            return -1
        return ans
    except Exception as e:
        print("category", e)
        return -1

def getCurPrice(date, symbol, flag=True):
    if(flag):
        date = date.split("-")
        if(len(date) == 1):
            date = date[0].split("/")
        date = datetime.datetime(int(date[2]),int(date[1]),int(date[0]) )
        
    minDate = datetime.datetime(2017,1,1)
    while(date > minDate ):
        try:
            YY = date.strftime("%Y")
            mon = date.strftime("%b").upper()
            dd = date.strftime("%d")
            df = pd.read_csv(f"C:\\Users\\garya\\ML projects\\Stock analysis\\datasets\\nsedataPrevious5years\\cm{dd}{mon}{YY}bhav.csv").set_index("SYMBOL")
            if(symbol not in df.index):
                return -1
            ans = df['CLOSE'][symbol]
            return ans
        except Exception as e:
            x = 1           
#           print(e)
                
        finally:
            date -= datetime.timedelta(days=1)
    return -1

def getHighLowValues(date, symbol):
#     print(symbol)
    high = getCurPrice(date,symbol)
    low = high
    date = date.split("-")
    if(len(date) == 1):
        date = date[0].split("/")
    date = datetime.datetime(int(date[2]),int(date[1]),int(date[0]) )
    for i in range(1,90):
        date += datetime.timedelta(days=1)
        val = float(getCurPrice(date,symbol,False))
        if(val == -1):
            continue
        high = max(high,val )
        low = min(low, val)
    
    return high,low
        

In [10]:
print(getCurPrice("28/02/2021", "3MINDIA"))
print(getHighLowValues("28/02/2021", "3MINDIA"))

22832.75
(30436.65, 22832.75)


In [11]:
quarter_result_dates.iloc[0]

index                    1
companyname    amjumbobags
companycode           JB03
nsesymbol          AMJUMBO
bsesymbol                 
                  ...     
Mar 1999                  
Sep 1997                  
Mar 1998                  
Mar 1997                  
Dec 1996                  
Name: 0, Length: 110, dtype: object

In [57]:
for i in range(len(quarter_result_dates)):
# for i in range(6,7):
    try: 
        date_row = quarter_result_dates.iloc[i]
        print(date_row)
        symbol = date_row['nsesymbol']
        companyname = date_row['companyname']
        companycode = date_row['companycode']
        columns = date_row.index
        fundamental_row = quarterly_df.loc[symbol]
#         print(fundamental_row)
        for qtr in quarterPairs:
            try:
                curQuarter = qtr[0]
                prevQuarter = qtr[1]
                
                if((curQuarter in columns)):
                    generateCategory(prevQuarter, curQuarter, fundamental_row, symbol)

            except Exception as e:
                print("For", e)
            
    except Exception as e:
        print("main", e)

index                    1
companyname    amjumbobags
companycode           JB03
nsesymbol          AMJUMBO
bsesymbol                 
                  ...     
Mar 1999                  
Sep 1997                  
Mar 1998                  
Mar 1997                  
Dec 1996                  
Name: 0, Length: 110, dtype: object
main 'AMJUMBO'
index                     2
companyname    abinfrabuild
companycode             I07
nsesymbol           ABINFRA
bsesymbol                  
                   ...     
Mar 1999                   
Sep 1997                   
Mar 1998                   
Mar 1997                   
Dec 1996                   
Name: 1, Length: 110, dtype: object
main 'ABINFRA'
index                            8
companyname    a2zinfraengineering
companycode                  AME02
nsesymbol                 A2ZINFRA
bsesymbol                 533292.0
                      ...         
Mar 1999                          
Sep 1997                          
Mar 1998     

index                       182
companyname    ajoonibiotechltd
companycode               ABL01
nsesymbol                AJOONI
bsesymbol                      
                     ...       
Mar 1999                       
Sep 1997                       
Mar 1998                       
Mar 1997                       
Dec 1996                       
Name: 83, Length: 110, dtype: object
main 'AJOONI'
index                      183
companyname    ajrinfratolling
companycode              GIP01
nsesymbol             AJRINFRA
bsesymbol             532959.0
                    ...       
Mar 1999                      
Sep 1997                      
Mar 1998                      
Mar 1997                      
Dec 1996                      
Name: 84, Length: 110, dtype: object
category 'Margin Dec 2022'
category must be real number, not str
category must be real number, not str
index                         186
companyname    akashinfraprojects
companycode                  AI76
nsesymbol     

Name: 207, Length: 110, dtype: object
main 'ASIAN-RE1'
index                    448
companyname    aslindustries
companycode             AI77
nsesymbol             ASLIND
bsesymbol                   
                   ...      
Mar 1999                    
Sep 1997                    
Mar 1998                    
Mar 1997                    
Dec 1996                    
Name: 208, Length: 110, dtype: object
main 'ASLIND'
index                       452
companyname    aspinwallcompany
companycode                AC27
nsesymbol             ASPINWALL
bsesymbol                      
                     ...       
Mar 1999                       
Sep 1997                       
Mar 1998                       
Mar 1997                       
Dec 1996                       
Name: 209, Length: 110, dtype: object
category 'Margin Dec 2022'
category must be real number, not str
category must be real number, not str
index                        456
companyname    assamcompanyindia
companycode    

index                                      614
companyname    bhagiradhachemicalsindustriesre
companycode                              BCI05
nsesymbol                              BCIL-RE
bsesymbol                             750708.0
                            ...               
Mar 1999                                      
Sep 1997                                      
Mar 1998                                      
Mar 1997                                      
Dec 1996                                      
Name: 306, Length: 110, dtype: object
main 'BCIL-RE'
index                    616
companyname    bclindustries
companycode             BC06
nsesymbol             BCLIND
bsesymbol           524332.0
                   ...      
Mar 1999                    
Sep 1997                    
Mar 1998                    
Mar 1997                    
Dec 1996                    
Name: 307, Length: 110, dtype: object
category 'Margin Dec 2022'
category must be real number, not str
category

Name: 416, Length: 110, dtype: object
category 'Margin Dec 2022'
index                                    837
companyname    computeragemanagementservices
companycode                              CAM
nsesymbol                               CAMS
bsesymbol                           543232.0
                           ...              
Mar 1999                                    
Sep 1997                                    
Mar 1998                                    
Mar 1997                                    
Dec 1996                                    
Name: 417, Length: 110, dtype: object
category 'Margin Dec 2022'
category must be real number, not str
category must be real number, not str
index                  840
companyname    canfinhomes
companycode            CFH
nsesymbol       CANFINHOME
bsesymbol         511196.0
                  ...     
Mar 1999        28-04-1999
Sep 1997                  
Mar 1998                  
Mar 1997                  
Dec 1996                  
Na

...

index              4790
companyname         srs
companycode         S17
nsesymbol        SRSLTD
bsesymbol      533569.0
                 ...   
Mar 1999               
Sep 1997               
Mar 1998               
Mar 1997               
Dec 1996               
Name: 2402, Length: 110, dtype: object
category 'Margin Dec 2022'
index                        4800
companyname    stampedecapitaldvr
companycode                 SCD01
nsesymbol                 SCAPDVR
bsesymbol                570005.0
                      ...        
Mar 1999                         
Sep 1997                         
Mar 1998                         
Mar 1997                         
Dec 1996                         
Name: 2403, Length: 110, dtype: object
main 'SCAPDVR'
index                     4801
companyname    stampedecapital
companycode               BS11
nsesymbol             STAMPEDE
bsesymbol             531723.0
                    ...       
Mar 1999            30-04-1999
Sep 1997            27-10

main 'TASTYBITE'
index                   5067
companyname    tatachemicals
companycode               TC
nsesymbol           TATACHEM
bsesymbol           500770.0
                   ...      
Mar 1999                    
Sep 1997                    
Mar 1998                    
Mar 1997                    
Dec 1996                    
Name: 2524, Length: 110, dtype: object
category 'Margin Dec 2022'
category must be real number, not str
category must be real number, not str
index                5068
companyname    tatacoffee
companycode          TC14
nsesymbol      TATACOFFEE
bsesymbol        532301.0
                  ...    
Mar 1999                 
Sep 1997                 
Mar 1998                 
Mar 1997                 
Dec 1996                 
Name: 2525, Length: 110, dtype: object
category 'Margin Dec 2022'
category must be real number, not str
category must be real number, not str
index                        5069
companyname    tatacommunications
companycode               

index                               5203
companyname    touchwoodentertainmentltd
companycode                          TEL
nsesymbol                      TOUCHWOOD
bsesymbol                               
                         ...            
Mar 1999                                
Sep 1997                                
Mar 1998                                
Mar 1997                                
Dec 1996                                
Name: 2605, Length: 110, dtype: object
category 'Margin Dec 2022'
index                             5204
companyname    tourismfinancecorpindia
companycode                        TFC
nsesymbol                      TFCILTD
bsesymbol                     526650.0
                        ...           
Mar 1999                              
Sep 1997                              
Mar 1998                              
Mar 1997                              
Dec 1996                              
Name: 2606, Length: 110, dtype: object
category 'Margi

main 'VEEKAYEM'
index                    5456
companyname    veerhealthcare
companycode              NI08
nsesymbol          NIYATILEAS
bsesymbol            511523.0
                    ...      
Mar 1999                     
Sep 1997                     
Mar 1998                     
Mar 1997                     
Dec 1996                     
Name: 2733, Length: 110, dtype: object
main 'NIYATILEAS'
index              5464
companyname      venkys
companycode         V03
nsesymbol       VENKEYS
bsesymbol      523261.0
                 ...   
Mar 1999               
Sep 1997               
Mar 1998               
Mar 1997               
Dec 1996               
Name: 2734, Length: 110, dtype: object
main 'VENKEYS'
index                         5465
companyname    venlonpolyesterfilm
companycode                    VPF
nsesymbol               VENLONPOLY
bsesymbol                 524038.0
                      ...         
Mar 1999                12-04-1999
Sep 1997                          

index                                    5706
companyname    zicomelectronicsecuritysystems
companycode                               ZSS
nsesymbol                               ZICOM
bsesymbol                            531404.0
                            ...              
Mar 1999                                     
Sep 1997                                     
Mar 1998                                     
Mar 1997                                     
Dec 1996                                     
Name: 2850, Length: 110, dtype: object
category 'Margin Dec 2022'
category must be real number, not str
index                     5707
companyname    zimlaboratories
companycode         ZIMLA54140
nsesymbol               ZIMLAB
bsesymbol             541400.0
                    ...       
Mar 1999                      
Sep 1997                      
Mar 1998                      
Mar 1997                      
Dec 1996                      
Name: 2851, Length: 110, dtype: object
category '

In [58]:
for x in dict1:
    if(len(dict1[x]) > 1 and len(dict1[x]) < 10):
        print(x, dict1[x])
    

[0.0, 1.0, 4.0, 0.0] {"['EXPLEOSOL', 'Sep 2022']", "['DSSL', 'Sep 2019']", "['AARVEEDEN', 'Mar 2019']"}
[0.0, 1.0, 6.0, -0.0] {"['MEGASTAR', 'Jun 2022']", "['AARVI', 'Sep 2022']", "['MOLDTKPAC', 'Dec 2017']"}
[-2.0, -1.0, 17.0, -1.0] {"['TATASTLLP', 'Jun 2018']", "['AARVI', 'Mar 2022']"}
[0.0, 1.0, 8.0, 0.0] {"['AARVI', 'Dec 2021']", "['MURUDCERA', 'Sep 2022']", "['VAKRANGEE', 'Sep 2022']", "['PPAP', 'Mar 2022']", "['OAL', 'Jun 2021']", "['TALBROAUTO', 'Sep 2022']"}
[1.0, 1.0, 7.0, 1.0] {"['TOTAL', 'Jun 2021']", "['AARVI', 'Sep 2021']"}
[-1.0, -0.0, -2.0, -1.0] {"['CYBERMEDIA', 'Mar 2019']", "['AARVI', 'Sep 2020']"}
[-2.0, -1.0, -1.0, -1.0] {"['JASCHIND', 'Jun 2022']", "['AARVI', 'Dec 2019']", "['BIMETAL', 'Jun 2022']", "['PIONEEREMB', 'Dec 2021']"}
[5.0, 3.0, 4.0, 2.0] {"['JASH', 'Sep 2018']", "['AARVI', 'Sep 2019']"}
[3.0, 1.0, -1.0, -1.0] {"['BIRLAMONEY', 'Jun 2018']", "['ARIHANTCAP', 'Jun 2021']"}
[3.0, 1.0, 1.0, 0.0] {"['MAJESAUTO', 'Jun 2022']", "['ALMONDZ', 'Dec 2019']", "['BIRL

In [ ]:
{"['GALAXYSURF', 'Dec 2018']", "['TIDEWATER', 'Jun 2022']"}

In [ ]:
marginBit, ebitdaBit,revenueBit, profitBit